# Python Workshop
# Session 3: Using Application Programming Interfaces

Stefan Scholz

In this third session we will learn how to **stream data** from internet services. For this purpose we will discuss how **data** from the **internet** can be used generally, and attempt it on an exemplary internet service.

## 3.1 Data Streams

In our context, a data stream is a sequence of information. While in a true data stream not all data is available at one point in time, in pseudo data streams the data is completely available but the data is usually so big that the hard drive or memory cannot cope with it. Depending on the type of data stream, it can contain various information, i.e. timestamps, attributes, raw data or processed data.

These information are streamed in a very broad range of applications. Below are a few examples.

| Application | Description |
| -------- | ------- |
| Fraud detection | detect patterns and anomalies in banking transactions with time, amount, location |
| Click analytics | profile advertisment, social media, websites by user behavior |
| News analytics | underpin trading strategy or business decision by automated text analysis on news |
| High-frequency trading | make automated trading decision in milliseconds based on live stock prices |

In order to get a data stream, most of the times either websites are used where the data comes straight from a data provider without being processed, or from APIs where the data is somehow modified, validated or cleaned by some data provider. What this exactly means and how we get access to these data streams, we will learn in today's session.


## 3.2 Iterators

But before we come to data streams from the internet, we should first learn how data streams are defined in Python. In fact, to work with changing and large data streams, Python offers `Iterator`.

In our first session, we got to know a similarly named construct called `Iterable`. However, a `Iterable` is a data structure, e.g. list, tuple, dictionary, whereas `Iterator` is more like a pointer which points on certain data. Technically, a `Iterator` is an object that can be iterated upon with the help of the methods `__iter__()` and `__next__()`. Actually most iterables have a build-in iterable to access them. But you can also create your own iterator object which implements the two methods above.

The big advantage about iterators is that with large amounts of data not all data need to be stored at once. For example, if we want to access all numbers from $0$ to $10^{50}$, we could try to store all numbers in a list, which will be for sure super large, but will probably not even fit into your memory. Instead we could use an iterator that creates each number after the other, such that only one number has to be stored at a time. This is a typical example where iterators are very helpful.

Let us create this iterator and return some numbers.

In [ ]:
class Number:
    """
    Class Number
    """

    def __iter__(self):
        self.next_number = 1
        return self

    def __next__(self):
        current_number = self.next_number
        self.next_number += 1
        return current_number

In [ ]:
# iterate over numbers
for number in Number():
    # print next number
    print(number)

    # stop iteration
    if number >= 3:
        break

In [ ]:
# define iterator for numbers
numbers = iter(Number())

# print first number
print(next(numbers))

# print second number
print(next(numbers))

# print third number
print(next(numbers))

## 3.3 Exceptions

Until now you have probably strumbled across several error messages when you wrote Python code. In general, these error messages are divided into two categories: First, there are syntax errors, which indicate that at some point in your code you used an invalid command, e.g. you forgot an indent or wrote a colon too much. The interpreter checks for these syntax errors before you code is actually executed. But we do not want to go into detail here. Instead, we want to discuss the second category of error messages. These error messages are problems which the interpreter encounters when it actually executes your code. These errors are also called exceptions. By default, they are fatal and stop your program immediately when the exception occurs.

In the following is a list of common exceptions:

| Exception | Cause |
| -------- | ------- |
| Attribute Error | Raised when attribute assignment or reference fails |
| Import Error | Raised when the imported module is not found |
| Index Error | Raised when index of a sequence is out of range |
| KeyError | Raised when a key is not found in a dictionary |
| Keyboard Interrupt | Raised when the user hits interrupt key(Ctrl + C or Delete) |
| Memory Error | Raised when an operation runs out of memory |
| Name Error | Raised when a variable is not found in local or global scope |
| Syntax Error | Raised by parser when syntax error is encountered |
| IndentationError | Raised when there is incorrect indentation |
| Type Error | Raised when a function or operation is applied to an object of incorrect type |
| Value Error | Raised when a function gets argument of correct type but improper value |
| Zero Division Error | Raised when second operand of division or modulo operation is zero |

If you are working with data streams, e.g. from websites and APIs, it is advisable that you take certain errors into account such that not the whole program aborts because of an unimportant detail in the data stream. Besides the data stream itself, there is an endless number of potential causes for errors.

In these cases, we wrap our code with a `try` statement, and catch a possible exception with an `except` statement. You can also catch multiple exceptions at the same time by adding underneath more `except` statements. If you want to have the respective message of the exception available in the `exception` block give it a variable name, like in the `with` statement. If you use a `finally` statement at the end of your `try` statement, the clause inside the `finally` statement will be executed last, whether or not the `try` statement raised an exception.

Let us catch some trivial exceptions.

In [ ]:
# error prone code
size = len(w)

In [ ]:
try:
    # error prone code
    size = len(x)
except NameError as e:
    # report name error
    print("Got error: {}".format(e))

Please keep in mind, however, that you should not abuse `try` statements to make poor code run, but only to deal with unavoidable problems. This is also the reason why we have not introduced exception handling earlier on.

## 3.4 REST APIs

For the purpose of communication between clients and servers, a wide range of application programming interfaces (APIs) has been developed. An API can appear in different forms, e.g. as web system, operating system or database system. However, in today's session we want to dicuss a specific type called REST APIs. REST is an acronym for REpresentational State Transfer Application Programming Interface. This form of interfaces defines a uniform and predefined set of stateless operations.

In this context, stateless means that in every request all information is sent and no previous request is considered. This ensures that the interfaces are fast, reliable and fault tolerant.

In particular, the client sends a request with a URL, with an endpoint, an access token and other parameters. You can imagine this as if you access a webpage. But an API is much more generic and abstract and allows for much more requests. The HTTP protocol is used to handle the requests listed below. We will also send requests using the package `requests`. This package is a very powerful package which does a lot of work for us in the background to send such requests. All we have to do is determine our request with a URL, header and parameters.

In the following is a list of the most common HTTP methods used together with REST APIs.

| Method | Description |
| -------- | ------- |
| GET | request data from the server |
| POST | sends data to the server |
| PUT | changes data on the server |
| DELETE | deletes existing data on the server |

For us, when we want to stream data, the first method `GET` will be particularly important. These requests will be processed by the server and then the client will receive a response according to the corresponding request, usually as HTML, XML or JSON. This response will be processed by the package `requests` again, which we can use to access the underlying data.

We will discuss the individual steps to work with a REST API in the following. Therefore, we will use as an example the [News API](https://newsapi.org/). This is a REST API that provides news headlines from 30,000 news sources worldwide. However, for copyright reasons, it does not provide the full texts of the articles but only the link to these articles. Basically, these steps can be implemented similarly for all REST APIs.

### Endpoints

REST APIs have so-called endpoints against which requests can be made. But one REST API can have multiple endpoints which all provide different information. However, if you work with a single REST API, then all its endpoints have the same URL as base. In this case, for the different endpoints only the suffix behind the base URL changes. For example, [News API](https://newsapi.org/) has in total three endpoints which provide different kinds of information. A detailed description of these endpoints is available in the [documentation](https://newsapi.org/docs/endpoints).

Below you find an overview of the different endpoints of [News API](https://newsapi.org/).

| Endpoint | Description |
| -------- | ------- |
| https://newsapi.org/v2/top-headlines | breaking news headlines |
| https://newsapi.org/v2/everything | recent news and blog articles |
| https://newsapi.org/v2/sources | available news sources |

### Parameters

Parameters are options you can pass with the endpoints which restrict what specific information you are interested in. Usually three different types of parameters are used. For all REST APIs, the documentation will tell you in specific which information you should pass in which parameter type.

The following list shows the different parameter types.

| Type | Description |
| -------- | ------- |
| Header | in request header, e.g. authorization |
| Query string | in request string after endpoint and question mark `?`, e.g. keywords |
| Request body | in request body usually as JSON, e.g. send data |

We will later combine the various parameters with the package `requests`. This package will take care of the appropriate handling of the different types of parameters. We will come to this when we actually make a request. Before that we should have a look at the parameters of [News API](https://newsapi.org/). In specific, we will only look at one endpoint of it which provides recent news and blog articles. You can find a detailed description for all parameters and endpoints in the [documentation](https://newsapi.org/docs/endpoints/).

Most REST APIs use an API key, also [News API](https://newsapi.org/). This API key should be passed in the headers of your requests. The following list shows the required parameters in the header for a request on the [News API](https://newsapi.org/).

| Header | Description |
| -------- | ------- |
| X-Api-Key | authentication with API key |

While this authentification method in the header is very common and widely used, the query parameters differ significantly from API to API. This is also because they depend on what information is available in the API. You can find a detailed description of the query parameters for the endpoint `everything` in the [documentation](https://newsapi.org/docs/endpoints/everything). In the following list we see a few query parameters which can be used together with the endpoint `everything`.

| Query string | Description |
| -------- | ------- |
| q | keywords or phrases to search for in the article title and body, e.g. `bitcoin` |
| qInTitle | keywords or phrases to search for in the article title only, e.g. `bitcoin` |
| sources | comma-seperated string of identifiers for the new sources or blogs you want headlines from (maximum 20), e.g. `the-new-york-times` |
| from | date and optional time for the oldest article allowed, e.g. `2019-10-17` |
| to | date and optional time for the newest article allowed, e.g. `2019-10-17` |
| language |  2-letter ISO-639-1 code of the language you want to get headlines, e.g. `de`, `en`, `es`, `fr` |
| sortBy | order to sort the articles in, e.g. `relevancy`, `publishedAt`  |

When we have prepared all the parameters, we are ready to make a request to a REST API.

### Requests

There are several ways to send a request to a REST API, among others you can do it with the package `requests`. As mentioned above, this package allows you to make HTTP requests very easily and quickly. It provides all functions and methods to write your parameters into requests, send you requests and work with your responses.

Let us first import the package or install it if necessary.

In [ ]:
import requests

In our example with the [News API](https://newsapi.org/) we are interested in getting recent news articles. The best way to understand requests is to prepare an exemplary request on some news articles.

Let us first define the URL with the endpoint in the variable `url`, then the headers in the variable `header` and the query strings in the variable `parameters`. Then we pass these variables into the function `requests.get()` to make the corresponding request. This request is best wrapped inside a `try` statement, e.g. to handle an HTTP error `requests.exceptions.HTTPError`. An exception will also be raised by the method `raise_for_status()` when the response contains an invalid status.

In [ ]:
# define token
token = "YOUR_API_KEY"

# define entrypoint
url = "https://newsapi.org/v2/everything"

# define header
header = {"X-Api-Key": token}

# define query strings
parameters = {"qInTitle": "hamas", "language": "en"}

try:
    # make request
    response = requests.get(url, params=parameters, headers=header)
    # check response
    response.raise_for_status()
except requests.exceptions.HTTPError as e:
    print("HTTPError: {}".format(e))
except Exception as e:
    print("Error: {}".format(e))

If the request works and throws no error, you can have a first look into the response with the attribute `content`.

In [ ]:
# inspect response
print(response.json())

<div class="alert alert-block alert-info">
    <b>Exercise</b>: Sign up for <a href="https://newsapi.org/register">NewsAPI</a>.
</div>

<div class="alert alert-block alert-info">
    <b>Exercise</b>: Request English articles from the BBC on Hamas from the <a href="https://newsapi.org/">News API</a>.
</div>

<div class="alert alert-block alert-info">
    <b>Exercise</b>: Inspect the response. Collect the titles and urls of the articles as lists.
</div>

### Module

For the News API there is a readily available Python module. The functions and methods for this module should mirror the endpoints specified by the News API documentation. After installation, import the client class into your project.

This module makes it much easier to get the news articles from News API, rather than manually writing the requests manually. But unfortunately not every API has its own Python module. In these cases we still need to make the requests.

Let us import the module.

In [ ]:
from newsapi import NewsApiClient

The client which takes care of the communication with the API can be initialized with the API key (same key as you used before).

Let us create a client.

In [ ]:
client = NewsApiClient(api_key="YOUR_API_KEY")

By calling on the client the method `get_everything()` we can make a request to the endpoint `everything` of the News API.

Let us make the same request as before.

In [ ]:
# make request and save response
response = client.get_everything(q="hamas", language="en", sources="bbc-news")

# show response
print(response)

Let us save the response in a JSON file to use it later on.

In [ ]:
import json

In [ ]:
with open("data/articles.json", "w") as file:
    json.dump(response, file)